# 第13章 类继承

## 13.1 一个简单的基类

### 13.1.1 派生一个类

`RatedPlayer`类声明为从`TableTennisClass`类派生而来：
```cpp
// RatedPlayer derives from the TableTennisPlayer base class
class RatedPlayer : public TableTennisPlayer {
...
};
```

冒号指出`RatedPlayer`类的基类是`TableTennisPlayer`类，上述特殊的声明头表明`TableTennisPlayer`是一个公有基类，这被称为公有派生。派生类对象包含基类对象。使用公有派生，基类的公有成员将成为派生类的公有成员；基类的私有部分也将成为派生类的一部分，但只能通过基类的公有方法和保护方法访问。

`RatedPlayer`对象将具有以下特征：
1. 派生类对象存储了基类的数据成员(派生类继承了基类的实现)
2. 派生类对象可以使用基类的方法(派生类继承了基类的接口)



### 13.1.2 构造函数：访问权限的考虑

派生类不能直接访问基类的私有成员，而必须通过基类方法进行访问。派生类的构造函数必须使用基类构造函数。

创建派生类对象时，程序首先创建基类对象。C++使用成员初始化列语法来完成这种工作。
```cpp
RatedPlayer::RatedPlayer(unsigned int r, const string & fn,
    const string & ln, bool ht) : TableTennisPlayer(fn, ln, ht)
{
    rating = r;
}
```

其中`:TableTennisPlayer(fn, ln, ht)`是成员初始化列表。它是可执行代码，调用`TableTennisPlayer`构造函数。

有关派生类构造函数的要点如下：
- 首先创建基类对象
- 派生类构造函数应通过成员初始化列表将基类信息传递给基类构造函数
- 派生类构造函数应初始化派生类新增的数据成员

释放对象的顺序与创建对象的顺序相反，即首先执行派生类的析构函数，然后自动调用基类的析构函数。

### 13.1.3 使用派生类

要使用派生类，程冠希必须要能够访问基类声明。可以将每个类放在独立的头文件中，但由于这两个类是相关的，所以把其类声明放在一起更合适。
**程序清单13.4 tabtenn1.h**


以下代码是两个类的定义
**程序清单13.5 tabtenn1.cpp**


### 13.1.4 派生类和基类之间的特殊关系
派生类与基类之间有一些特殊关系，其中之一是**派生类对象可以使用基类的方法，条件是方法不是私有的。**

另外两个重要关系是：
1. 基类指针可以在不进行显式类型转换的情况下指向派生类对象。
2. 基类引用可以在不进行显式类型转换的情况下引用派生类对象。

```cpp
RatedPlayer rplayer1(1140, "Mallory", "Duck", true);
TableTennisPlayer * pt = &rplayer; // rule 1
TableTennisPlayer & rt = rplayer;  // rule 2
pt->Name();    // invoke Name() with pointer
rt.Name();     // invoke Name() with reference
```

但是不能将基类对象的地址赋给派生类引用和指针：
```cpp
TableTennisPlayer player("Betsy", "Bloop", true);
RatedPlayer & rr = player;    // NOT ALLOWED
RatedPlayer * pr = player;    // NOT ALLOWED
```

---

## 13.2 继承：is-a 关系

C++有3种继承方式
1. 公有继承：建立一种 is-a 关系，例如Fruit类派生出Banana类，Banana类具有Fruit的所有数据成员，并且Banana还可以添加自己的新特性。
2. 保护继承：
3. 私有继承

---

## 13.3 多态公有继承

**多态（具有多种形态）**-- 希望同一个方法在派生类和基类中的行为是不同的。

有两种重要的机制可以用于实现多态公有继承：
1. 在派生类中重新定义基类方法
2. 使用虚方法

### 13.3.1 开发Brass类和BrassPlus类

**程序清单13.7 brass.h**

在程序13.7中，派生类 BrassPlus 使用了 virtual关键字，差别入下：
    1. 如果没有使用关键字 virtual，程序将根据 引用类型 或 指针类型 选择方法；
    2. 如果使用了 virtual，程序将根据 引用或指针指向的对象的类型 来选择方法。
    
```cpp
Brass fom("Donminic Banker", 1122, 21);
BrassPlus dot("DD", 12, 21)
Brass & b1_ref = dom;
Brass & b2_ref = dot;

// 1. 如果ViewAcct()不是虚的
// behavior with non-cirtual ViewAcct()
// method chosen according to reference type
b1_ref.ViewAcct();    // use Brass::ViewAcct()
b2_ref.ViewAcct();    // use Brass::ViewAcct()

// 2. 如果ViewAcct()是虚的
b1_ref.ViewAcct();    // use Brass::ViewAcct()
b2_ref.ViewAcct();    // use BrassPlus::ViewAcct()
```

由此可见，虚函数特别方便。因此，经常在基类中将派生类会重新定义的方法声明为虚方法。方法在基类中被声明为虚的之后，它在派生类中将自动成为虚方法。

#### 1. 类实现
**程序清单 13.8 brass.cpp**

#### 4. 为何需要虚析构函数
如果析构函数是虚的，将调用对应对象类型的析构函数。因此，当基类和子类的析构函数实现不同时，则必须要有一个虚析构函数。

---

## 13.4 静态联编和动态联编

**联编(binding)：**将源代码中的函数调用解释为执行特定的函数代码块被称为函数名联编(binding).
- 静态联编(static binding)：又称为早期联编(early binding)，在编译过程中进行联编
- 动态联编(dynamic binding)：又称为晚期联编(late binding)，在程序运行时选择正确的虚函数，进行联编。

### 13.4.1  指针和引用类型的兼容性
在C++中，动态联编与通过指针和引用调用方法相关。通常C++不允许将一种类型的地址赋给另一种类型的指针，也不允许一种类型的引用指向另一种类型。
```cpp
double x = 2.5;
int * pi = &x;    // invalid assignment, mismatched pointer type
long & ri = x;    // invalid assignment, mismatched reference type
```

但是基类和子类是可以的，指向基类的引用或指针可以引用派生类对象，而不必进行显式类型转换。
```cpp
BrassPlus dilly ("Annie Dill", 49322, 303);
Brass * pb = &dilly;  // ok
Brass & rb = dilly;   // ok
```


### 13.4.2 虚成员函数和动态联编


将派生类引用或指针转换为基类引用或指针被称为向上强制转换(upcasting)，

相反的过程--将基类指针或引用转换为派生类指针或引用--称为向下强制转换(downcasting)。如果不适用显示类型转换，则向下强制转换是不允许的。

#### 虚函数的工作原理
编译器处理虚函数的方法：给每个对象添加一个隐藏成员。该隐藏成员保存了一个指向函数地址数组的指针。这种数组称为虚函数表(virtual function table, vtbl)。虚函数表中存储了为类对象进行声明的虚函数的地址。

例如：基类队形包含一个指针，该指针指向基类中所有的虚函数的地址表。派生类对象将包含一个指向独立地址表的指针。如果派生类提供了虚函数的新定义，该虚函数表将保存新函数的地址；如果派生类没有重新定义虚函数，该vtbl将保存函数原始版本的地址。如果派生类定义了新的虚函数，则该函数的地址也将被添加到vtbl中。

### 13.4.3 有关虚函数的注意事项

#### 1. 构造函数
构造函数不能是虚函数。派生类有自己的构造函数，而不是继承基类的构造函数。即使派生类没有定义构造函数，默认的构造函数也会在内部调用基类的构造函数，这不是虚函数。

#### 2. 析构函数
析构函数应当是虚函数，除非类不用做基类。

#### 3. 友元
右元不嫩是虚函数，因为友元不是类成员，只有成员才能是虚函数。

#### 4. 没有重新定义
如果派生类没有重新定义函数，将使用该函数的基类版本。如果派生类位于派生链中，则将使用最新版本的虚函数，例外的情况是基类版本是隐藏的。

#### 5. 重新定义将隐藏方法
假设创建了如下代码：
```cpp
class Dwelling {
public:
    virtual void showperks(int a) const;
...
};

class Hovel : public Dwelling {
public:
    virtual void showperks() const;
...
};
```

这将导致问题，可能会出现类似下面的编译警告：
```
Warning: Hovel::showperks(void) hides Dwelling::showperks(int)
```

也可能不会出现警告。但是代码如下会有问题:
```cpp
Hovel trump;
trump.showperks();      // valid
trump.showperks(3);     // invalid
```

---

## 13.5 访问控制:  protected

关键字protected与private相似，在类外只能用公有类成员来访问protected部分的类成员。
protected与private的差别只有在基类派生的类中才能表现出来，派生类的成员可以直接访问基类的protected成员，但不能直接访问基类的private成员。

---

## 13.6 抽象基类

ABC -- Abstract Base Classs

C++通过使用纯虚函数(pure virtual function)提供未实现的函数。纯虚函数声明的结尾处为=0

```cpp
class BaseEllipse // abstract base class
{
private:
    double x;      // x coordinate of center
    double y;      // y coordinate of center
    
public:
    BaseEllipse(double x0 = 0, double y0 = 0) : x(x0),y(y0) {}
    virtual ~BaseEllipse() {}
    void Move(int nx, ny) { x = nx; y = ny; }
    virtual double Area() const = 0; // a pure virtual function
    ...
}
```

当类声明中包含纯虚函数时，则不能创建该类的对象。包含纯虚函数的类只用作基类。要成为真正的ABC，必须至少包含一个纯虚函数。原型中的=0使虚函数成为纯虚函数。虽然这里的Area()没有定义，但是C++允许纯虚函数有定义。

如股票，所有的基类方法都与Move()一样，那么则可以在基类中定义，但仍需要将这个类声明为抽象的。在这种情况下，可以将原型声明为虚的：
```cpp
void Move(int nx, ny) = 0;
void BaseEllipse::Move(int nx, ny) { x = nx; y = ny; }
```

虽然不能创建BaseEllipse对象，但是可以用BaseEllipse指针数组同时管理这两种对象。像Circle和Ellipse这样的类有时被称为具体(concrete)类。

### 13.6.1 应用 ABC 概念


### 13.6.2 ABC理念

---

## 13.7 继承和动态内存分配

### 13.7.1 第一种情况：派生类不使用new

### 13.7.2 第二种情况：派生类使用new

### 13.7.3 使用动态内存分配和友元的继承示例

---

## 13.8 类设计回顾

### 13.8.1 编译器生成的成员函数

### 13.8.2 其他的类方法

### 13.8.3 公有继承的考虑因素

### 13.8.4 类函数小结

---